# Google Colab (Skip locally)

In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/IFT6135/HW3_2025/src" #@param {type:"string"}
!ln -Ts "$folder" /content/src 2> /dev/null

# Add the assignment folder to Python path
if '/content/src' not in sys.path:
  sys.path.insert(0, '/content/src')

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

The system cannot find the path specified.
C:\Users\fwilh\AppData\Local\Temp\ipykernel_7844\128898963.py:21: UserWarning: CUDA is not available.
  warnings.warn('CUDA is not available.')


# Imports & Setup

In [2]:
%matplotlib inline
import warnings

from q1_train_vae import experiment1
from q2_trainer_ddpm import experiment2
from q3_trainer_cfg import experiment3

from shutil import make_archive
import os

models = ['VAE', 'DDPM', 'CFG']

usage: ipykernel_launcher.py [-h] [--batch-size N] [--epochs N] [--no-cuda]
                             [--no-mps] [--seed S] [--log-interval N]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\fwilh\AppData\Roaming\jupyter\runtime\kernel-v3bc677202c228232c59a198ed04bbbfab81676fec.json


SystemExit: 2

C:\Users\fwilh\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Experiments

## Experiment 1 (VAE)

In [ ]:
vae_trainer = experiment1()

In [ ]:
# TODO make_archive('experiment1', 'zip', 'logs/experiment1')

## Experiment 2 (Scaling Data Size : Training Ratio)

In [ ]:
ddpm_trainer = experiment2()

In [ ]:
# TODO make_archive('experiment2', 'zip', 'logs/experiment2')

## Experiment 3 (Scaling Data Size : P)

In [ ]:
cfg_trainer = experiment3()

In [ ]:
# TODO make_archive('experiment3', 'zip', 'logs/experiment3')